# PHYS465: introduction to markov chain monte-carlo

There are plenty of libraries that focus on MCMC in python. In the exercise we will develop our own, and then use existing tools to fit a cosmological model. 

The key libraries for this week are `emcee` (https://emcee.readthedocs.io/en/stable), `pymc` (https://www.pymc.io/welcome.html) and `corner` (https://corner.readthedocs.io/en/latest/). These libraries perform different tasks. In particular:
   * `emcee` is a MCMC sampler.
   * `pymc` is a Bayesian model builder (similar to, but more advanced than `emcee`)
   * `corner` is a data visualisation tool to show the results from MCMC fitting
   * `ArviZ` is a visualisation tool for Bayesian statistics
   * _NB_: Plenty of other librarries exist: as always, google is your friend. 
***

## A key aside: installing new libraries into colab

Google colab comes with a large array of libraries already installed. Should we need to install a new library we can use the command:
  * `%pip install {library}`
  * e.g. `> %pip install emcee`

If you are using `jupyter-notebooks` then the same command should work. It may require you to restart your kernel though. 

In [1]:
#%pip install emcee

***

## Introduction to MCMC: Posterior Calculation

Recall from the lectures that : $\text{posterior}=\text{likelihood} \times \text{prior}$
 * In this formula, the prior encodes our information _prior_ to collecting the data, so is independent of the data
 * i.e. $\text{prior} = f(\theta)$.
 * In many cases an assumption of a `uniform` or flat prior is taken: i.e. all values in parameter space (within a given bound) are equally likely to be true.

 * The likelihood depends on the model being tested.
    * $\mathcal{L}(\theta | x,y) = \left(\sum_{i=0}^{N}(2\pi\sigma_i^2)^{-N/2} \exp(-\frac{1}{2\sigma_i^2}\left(\frac{y - f(y,*\theta)}{\sigma_i}\right)^2)\right)$
    * $\rightarrow \ln \mathcal{L}(\theta | x,y) = -0.5 \times \left( N\times\ln(2\pi) + \sum_{i=0}^{N}2\times\ln(\sigma_i) + \sum_{i=0}^{N} \left(\frac{y - f(y,*\theta)}{\sigma_i}\right)^2\right)$  

  * NB, as we will see with `pymc` a fully Bayesian model will include an estimate of the global noise properties of the data as part of the model.
*** 

## MCMC : Metropolis Hastings Algorithm
MCMC is a sampling algorithm. It generates samples from what we refer to as a posterior (which is basically a function).
We use this technique to numerically solve complex problems by taking a random walk through parameter space.

The Metropolis-Hastings algorithm allows you to create a chain of evaluations of your function, which doesn’t depend on the initial conditions: only on the previous evaluation. In general our `walker` will walk towards areas of higher probability, but occassionally we will move to lower probability regions. the walker moves in steps that are a sphere in parameter space. 

As a result we can efficienty sample high probability regions without having to know where they are. This makes it far more efficient than grid search approaches. The only requirement is that we must be able to evaluate the posterior at a position $\theta$.
***

**The algorithm is:**
>1. pick an initial value of $\theta$ ($\theta_0$) and evaluate it as above. Add this to a stored “chain” of values
>
>2. pick a new value of $\theta$ ($\theta_1$), from _the proposal pdf_.
    * _Hint_: this is a Gaussian pdf of width $\sigma$ centered on $\theta_0$
>
>3. pick a number ($r$) from a uniform distribution between [0,1]
>
>4. for ($\theta_0,\theta_1$) calculate the posterior for both events $f(\theta_0)$, $f(\theta_1)$
>
>5. if $f(\theta_1)/f(\theta_0)>r$ then replace $\theta_0$ with $\theta_1$ (i.e. move to $\theta_1$) and add it to the chain
>
>6. otherwise, remain at $\theta_0$ and store that value t othe chain
>
>7. repeat for $N$ realisations

***

### Log acceptance
Above we calculated the posterior for both $\theta_0$ and $\theta_1$ and measured the ratio between them. 

It is ofter easlier to make this test in **ln posterior** space. In this case, rather than calculating $f(\theta_1)$ and $f(\theta_0)$ we calculate $\ln f(\theta_1)$ and $\ln f(\theta_0)$ and test for the following condition: 
 * $\ln f(\theta_1)$-$\ln f(\theta_0)>\ln(r)$
***

### Posteriors in Log Space

Recall from the lectures that : $\text{posterior}=\text{likelihood} \times \text{prior}$
 * For the `ln posterior` this becomes : $\ln\text{posterior}=\ln\text{likelihood} + \ln\text{prior}$
 * _NB_: recall that the prior is independent of our dataset so acts as an additive constant to this formula, if the prior is uniform.
***

## Introducing `emcee`

`emcee` is a powerful, easy to use and fast MCMC sampler. The only real requirements to produce chains is to write a function that calculates the `ln posterior` given a point in parameter space. With this function (`calc_ln_posterior()`) it is trivial to produce an MCMC run. 

This code extends beyond the simple Metropolis Hastings algorithm by allowing for **multiple walkers** where each walker journeys across parameter space based upon the movement of all other walkers.
 * This is known as `Ensemble sampling`

To run `emcee` we must given each walker a different starting position. e.g.

>`initial_guess = 4, 3` # Given the parameters of interest what is a good starting point. 
>
>`n_walkers, n_dim = 30, 2` # n_dim is the number of variables to consider
>
>`pos = initial_guess + 1e-3*np.random.randomn(n_walkers, n_dim)` # sample in a sphere around the best-fit. 1e-3 is an approximate uncertainty
>
>`sampler = emcee.EnsembleSampler(n_walkers, n_dim, calc_lc_posterior, args=(x,y,yerr))`
>
>`sampler.run_mcmc(pos, 5000, progress=True)` # for each walker run 5000 steps

Having run multiple chains, we can then combine them with: 
>`flat_samples = sample.get_chain(discard=100, thin=15, flat=True)` # `discard` sets the burn-in, `thin` means use only 1/15 of the samples.
***

## Introducing `pymc`

`pymc` is a very powerful Bayesian model builder and sampler. Unlike `emcee` which relies on the user providing a function to calculate the `log_posterior`, `pymc` takes as input a Bayesian model, which is then used to calculate the `posterior`. 

In this way it is more complicated to use than `emcee` but is a much more powerful tool. 

To run `pymc` we must first build a model. e.g.: 
>`import pymc as pm`
>
>`with basic_model`:
>
>    \# Set the names and priors for our model parameters
>
>`    inter = pm.Uniform("inter", lower=4, upper=5)`
>
>`    slope = pm.Uniform("slope", lower=1.1, upper=1.5)`
>
>`    sigma = pm.HalfNormal("sigma", sigma=1)` # A parameter that describes the average noise properties of the data. 
>
>    \# Define our model
>
>`    mu = inter + slope * x`
>
>    \# Define the likelihood
>
>`    y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)` # The values of y depend on the model and the noise

To sample this parameter space we can: 

>`with basic_model:`
>
>    \# draw 1000 posterior samples
>
>`    idata = pm.sample(1000)`

We can then plot and show the results using `Arviz`:

>`import arviz as az`
>
>`az.plot_trace(idata, combined=True)`
>
>`az.summary(idata, round_to=2)`

***